### Custom plugin to query company data that is obtained from an API

Make sure that the CompanyData Service is up and running before running this plugin. The CompanyData Service is a simple REST API that provides company data. 
```bash
cd "dotnet\notebooks\CompanyDataService\CompanyDataService"
dotnet run
```


In [ ]:
// Initialization and loading of modules

#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.19.0-preview"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core,1.20.0-alpha"
#r "nuget: Azure.Identity"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Azure.Identity;
using System.Net.Http;
using System.Threading;

In [14]:
// Custom HttpClient handler so we can log the input/output of the requests to the LLM API.
public class LoggingHandler : DelegatingHandler
{
    public LoggingHandler(HttpMessageHandler innerHandler)
        : base(innerHandler)
    {
    }

    protected override async Task<HttpResponseMessage> SendAsync(HttpRequestMessage request, CancellationToken cancellationToken)
    {
        Console.WriteLine("Request:");
        var r = request.ToString();
        var ra = r.Split("\n");
        foreach (var l in ra)
        {
            if (l.Contains("Authorization"))
            {
                Console.WriteLine("  Authorization: [REDACTED]");
            }
            else
            {
                Console.WriteLine(l);
            }
        }        
        if (request.Content != null)
        {
            Console.WriteLine(await request.Content.ReadAsStringAsync());
        }
        Console.WriteLine();

        HttpResponseMessage response = await base.SendAsync(request, cancellationToken);

        Console.WriteLine("Response:");
        Console.WriteLine(response.ToString());
        if (response.Content != null)
        {
            Console.WriteLine(await response.Content.ReadAsStringAsync());
        }
        Console.WriteLine();

        return response;
    }
}

In [15]:
// Create instance of HttpClient with our logging handler.
HttpClient oaiCLient = new HttpClient(new LoggingHandler(new HttpClientHandler()));
oaiCLient.Timeout = TimeSpan.FromMinutes(5);
var builder = Kernel.CreateBuilder();

// EntraID authentication options.
DefaultAzureCredentialOptions defaultAzureCredentialOptions = new DefaultAzureCredentialOptions 
{ 
    ExcludeSharedTokenCacheCredential = true, 
    ExcludeEnvironmentCredential = true, 
    ExcludeAzurePowerShellCredential = true, 
    ExcludeInteractiveBrowserCredential = true, 
    ExcludeVisualStudioCredential = true, 
    ExcludeManagedIdentityCredential = true, 
    ExcludeVisualStudioCodeCredential = true, 
    ExcludeAzureCliCredential = false // Only use az login credentials
};

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
useAzureOpenAI = true;
bool useLocalLLM = false;
Uri localLLMEndpoint = new Uri("http://localhost:11434/v1");
string localLLMModel = "phi3:latest";
#pragma warning disable SKEXP0010
AzureOpenAIPromptExecutionSettings pes = new AzureOpenAIPromptExecutionSettings { 
    MaxTokens = 1024, 
    Temperature = 0.001, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0 
    };  

// Initalize history
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history,
    ["promptExecutionSettings"] = pes
};

// Attach custom HttpCLient to the OAI connectors.
if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, credentials: new DefaultAzureCredential(defaultAzureCredentialOptions), httpClient: oaiCLient);
else if (useLocalLLM)
    builder.AddOpenAIChatCompletion(modelId:localLLMModel, apiKey:null, endpoint:localLLMEndpoint, httpClient: oaiCLient);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

In [16]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060
// Create a planner instance
var planner = new HandlebarsPlanner();

In [17]:
using System.ComponentModel;

using System;
using System.Collections.Generic;
using System.IO;
using System.Net.Http;

// Define the CompanyRecord class to hold the company data.
public record CompanyRecord
{
    public int Id { get; set; }
    public int Rank { get; set; }
    public string Name { get; set; }
    public string Industry { get; set; }
    public string City { get; set; }
    public string State { get; set; }
    public string Zip { get; set; }
    public string Website { get; set; }
    public int Employees { get; set; }
    public decimal RevenueInMillions { get; set; }
    public decimal ProfitInMillions { get; set; }
    public decimal ValuationInMillions { get; set; }
    public string Ticker { get; set; }
    public string CEO { get; set; }
}

// CompanyData plugin
public class CompanyDataPlugin
{
    string companyDataServiceApiEndpoint = " http://localhost:5232";
    HttpClient _httpClient = new HttpClient();

    // These descriptions are important and need to be as precise as possible as the Planner 
    // will use them as prompts to generate the HandleBar template.
    [KernelFunction("find_company")]
    [Description("Find a company name given a ticker symbol or description of the company.")]
    [return: Description("A unique ID to the company. If the ID returned is -1, that means the company info could not be found - in which caswe the user should be informed about it.")]
    public async Task<int> GetCompanyNameAsync(string CompanyText)
    {
        // Call the API to find/validate the company name.
        return await Task.Run(() =>
        {
            var url = companyDataServiceApiEndpoint + "/company_data/find/" + CompanyText;
            var r = _httpClient.GetAsync(url).Result;   
            if (r.StatusCode != System.Net.HttpStatusCode.OK)
            {
                return -1;
            }
            return int.Parse(r.Content.ReadAsStringAsync().Result);
        });
    }

    [KernelFunction("get_company_data")]
    [Description("Given the unique ID of a company, returns key data such as rank, industry, ticker symbol, address, website, valuation, profit, revenue, number of employees and CEO.")]
    [return: Description("The details of the company")]
    public async Task<CompanyRecord> GetCompanyDataAsync(string Id)
    {
        // Call the API to get the details of the company based on the ID.
        return await Task.Run(() =>
        {
            var url = companyDataServiceApiEndpoint + "/company_data/" + int.Parse(Id);
            var r = _httpClient.GetAsync(url).Result;
            var json = JsonDocument.Parse(r.Content.ReadAsStringAsync().Result);
            var root = json.RootElement;
            return new CompanyRecord
            {
                Id = root.GetProperty("id").GetInt32(),
                Rank = root.GetProperty("rank").GetInt32(),
                Name = root.GetProperty("name").GetString(),
                Industry = root.GetProperty("industry").GetString(),
                City = root.GetProperty("city").GetString(),
                State = root.GetProperty("state").GetString(),
                Zip = root.GetProperty("zip").GetString(),
                Website = root.GetProperty("website").GetString(),
                Employees = root.GetProperty("employees").GetInt32(),
                RevenueInMillions = root.GetProperty("revenueInMillions").GetDecimal(),
                ProfitInMillions = root.GetProperty("profitInMillions").GetDecimal(),
                ValuationInMillions = root.GetProperty("valuationInMillions").GetDecimal(),
                Ticker = root.GetProperty("ticker").GetString(),
                CEO = root.GetProperty("ceo").GetString()
            };
        });
    }
}


In [18]:
// Financial Math Plugin

public class FinancialMathPlugin
{
    // These descriptions are important and need to be as precise as possible as the Planner 
    // will use them as prompts to generate the HandleBar template.
    [KernelFunction("get_profit_margin")]
    [Description("Given the profit in millions and the revenue in millions, returns the profit margin in percentage.")]
    [return: Description("The profit margin in percentage")]
    public double GetProfitMargin(double profitInMillions, double revenueInMillions)
    {
        return profitInMillions / revenueInMillions * 100;
    }

    [KernelFunction("get_valuation_to_revenue")]
    [Description("Given the valuation in millions and the revenue in millions, returns the valuation to revenue ratio.")]
    [return: Description("The valuation to revenue ratio")]
    public double GetValuationToRevenueRatio(double valuationInMillions, double revenueInMillions)
    {
        return valuationInMillions / revenueInMillions;
    }

    [KernelFunction("get_valuation_to_profit")]
    [Description("Given the valuation in millions and the profit in millions, returns the valuation to profit ratio.")]
    [return: Description("The valuation to profit ratio")]
    public double GetValuationToProfitRatio(double valuationInMillions, double profitInMillions)
    {
        return valuationInMillions / profitInMillions;
    }

    [KernelFunction("get_valuation_to_employee")]
    [Description("Given the valuation in millions and the number of employees, returns the valuation to employee ratio.")]
    [return: Description("The valuation to employee ratio")]
    public double GetValuationToEmployeeRatio(double valuationInMillions, int employees)
    {
        return valuationInMillions / employees;
    }
}

In [19]:
#pragma warning disable SKEXP0050

//Add the plugins to the kernel
using Microsoft.SemanticKernel.Plugins.Core;
kernel.ImportPluginFromType<CompanyDataPlugin>();
kernel.ImportPluginFromType<FinancialMathPlugin>();


In [ ]:
#pragma warning disable SKEXP0060

// Question to ask the planner
var ask = "What is the valuation to profit ratio for Fedex?";

// Generate Handlebar template.
var plan = await planner.CreatePlanAsync(kernel, ask, arguments);

Console.WriteLine("Plan:\n");
Console.WriteLine(plan);

In [ ]:
#pragma warning disable SKEXP0060

// Execute Handlebar template.
var planResult = await plan.InvokeAsync(kernel);

Console.WriteLine("Plan results:\n");
Console.WriteLine(Utils.WordWrap(planResult.ToString(), 100));

In [ ]:
#pragma warning disable SKEXP0060

// Ask questions in a loop like a chatbot, with support for chat history.
ask = String.Empty;
while (ask != "exit")
{
    ask = await InteractiveKernel.GetInputAsync("Please ask your question about the company data");
    if (ask == "exit") continue;
    Console.WriteLine("Asking: " + ask);
    plan = await planner.CreatePlanAsync(kernel, ask, arguments);
    planResult = await plan.InvokeAsync(kernel);
    Console.WriteLine(Utils.WordWrap(planResult.ToString(), 100));
    history += $"\nUser: {ask}\nAI: {planResult.ToString()}\n";
    arguments["history"] = history;
}
